In [1]:
import os, time
import h5py
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
import tensorflow.keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import load_model

#K.set_image_dim_ordering('th')
tensorflow.keras.backend.set_image_data_format('channels_first')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
      # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
# path to the model files.
model_save_path = '/mnt/sdb1/zhuweilun/FASNet/weights/160_epoch80_casia_fasd_new.h5'

# Train

In [4]:
from tensorflow.keras.applications.vgg16 import VGG16

# dimensions of images. (less than 224x 224)
img_width, img_height = (160,160)

# nuumber of layers to freeze
nFreeze = 11

## Replayattack ##
#train_data_dir = '/home/mike/FASNet/replayattack/train/'
#validation_data_dir = '/home/mike/FASNet/replayattack/valid/'

## CASIA ##
train_data_dir = '/mnt/sdb1/zhuweilun/FASNet/casia_fasd/train/'
nb_train_samples = 9039
nb_epoch = 80

def get_tr_vgg_model(img_width, img_height):
    
    # build the VGG16 network
    model = VGG16(weights='imagenet', include_top=False, input_shape = (3,img_width,img_height))
    
    #freeze layers
    for layer in model.layers[:nFreeze]:
        layer.trainable = False
        
    #model.summary()
    #print(len(model.layers))
    
    return model

def add_top_layers(model):
    
    combine_model = Sequential()
    combine_model.add(model)
    
    top_model = Sequential()
    top_model.add(Flatten(input_shape=model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    #top_model.add(Dense(2, activation='softmax'))
    top_model.add(Dense(1, activation='sigmoid'))

    # add the model on top of the convolutional base
    combine_model.add(top_model)
    
    return combine_model

def run_train(model):
    
    start_time = time.time()

    # compile model
    model.compile(loss='binary_crossentropy',
    #model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6),
     #         metrics=['categorical_accuracy'])
              metrics=['accuracy'])
    
    print('Model Compiled.')
    
    train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip = True,
            fill_mode='nearest',
            data_format="channels_first")

#     test_datagen = ImageDataGenerator(rescale=1./255,
#                                       data_format="channels_first")

    train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_height, img_width),
            batch_size=200,
            #class_mode='categorical')
            class_mode='binary')
    
    label_map1 = (train_generator.class_indices)
    print(label_map1)

#     validation_generator = test_datagen.flow_from_directory(
#             validation_data_dir,
#             target_size=(img_height, img_width),
#             batch_size=200,
#             #class_mode='categorical')
#             class_mode='binary')
    
    #label_map2 = (validation_generator.class_indices)
    #print(label_map2)

    print('\nFine-tuning top layers...\n')

#     earlyStopping = callbacks.EarlyStopping(monitor='val_loss',
#                                            patience=10, 
#                                            verbose=0, mode='min', restore_best_weights=True)

    #fit model
    #model.fit_generator(
    #       train_generator,
    #       callbacks=[earlyStopping],
    #      use_multiprocessing = False,
    #       steps_per_epoch=nb_train_samples/100,
    #      epochs=nb_epoch,
    #       validation_data=validation_generator,
    #       validation_steps=nb_validation_samples/100)
    
#     model.fit(
#            train_generator,
#            callbacks=[earlyStopping],
#           use_multiprocessing = False,
#            steps_per_epoch=nb_train_samples/200,
#           epochs=nb_epoch,
#            validation_data=validation_generator,
#            validation_steps=nb_validation_samples/200)

    model.fit(
            train_generator,
            use_multiprocessing = False,
            steps_per_epoch=nb_train_samples/200,
            epochs=nb_epoch)


    #model.save_weights(top_model_weights_path)
    model.save(model_save_path)
    
    print('\nDone fine-tuning, have a nice day!')
    print("\nExecution time %s seconds" % (time.time() - start_time))

In [ ]:
if __name__ == "__main__":
   
    vgg16_tr_model = get_tr_vgg_model(img_width, img_height)
    vgg16_tr_model = add_top_layers(vgg16_tr_model)
    vgg16_tr_model.summary()
    
    # fine-tuning the model 
    run_train(vgg16_tr_model)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512, 5, 5)         14714688  
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 3277313   
Total params: 17,992,001
Trainable params: 16,256,513
Non-trainable params: 1,735,488
_________________________________________________________________
Model Compiled.
Found 9039 images belonging to 2 classes.
{'attack': 0, 'real': 1}

Fine-tuning top layers...

Epoch 1/80
46/45 [==============================] - 43s 929ms/step - loss: 3.6996 - accuracy: 0.7554
Epoch 2/80
46/45 [==============================] - 43s 934ms/step - loss: 3.7065 - accuracy: 0.7597
Epoch 3/80
46/45 [==============================] - 43s 937ms/step - loss: 3.7065 - accuracy: 0.7597
Epoch 4/80
46/45 [==============================] - 43s 935ms/step - loss: 3.7065 - 